# Tabular Playground Series - Feb 2021

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt 
import seaborn as sns
%matplotlib inline
#plt.style.use('seaborn') #ggplot

from sklearn.linear_model import LinearRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, mean_absolute_error
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error
import csv

## Import Data

In [ ]:
Data = pd.read_csv('../input/tabular-playground-series-feb-2021/train.csv', na_values = "?")
#Data copy contain cat + con values
Data_copy = pd.read_csv('../input/tabular-playground-series-feb-2021/train.csv', na_values = "?")
test = pd.read_csv('../input/tabular-playground-series-feb-2021/test.csv', na_values = "?")
submission = pd.read_csv('../input/tabular-playground-series-feb-2021/sample_submission.csv', na_values = "?")

## Explore Data

In [ ]:
Data.info()

In [ ]:
Data.describe()

In [ ]:
Data.head()

In [ ]:
Data.tail()

In [ ]:
for i in Data:
    if(i.find("cat")!= -1):    
        print(i , Data[i].unique())

In [ ]:
#Encoding Cat Data
for i in Data:
    if(i.find("cat")!= -1):
        Encoding = OrdinalEncoder()
        Data[i]=Encoding.fit_transform(Data[i].values.reshape(-1,1))

In [ ]:
Data.corr()

In [ ]:
plt.figure(figsize=(20,15))
sns.heatmap(Data.corr(), annot=True)

In [ ]:
Data.corr().unstack()

In [ ]:
feature_corr = Data.corr().unstack().sort_values()
feature_corr

In [ ]:
print(feature_corr[(abs(feature_corr)>0.5) & (abs(feature_corr)<1)].drop_duplicates())

In [ ]:
high_corr_df = pd.DataFrame(feature_corr[(abs(feature_corr)>0.5) & (abs(feature_corr)<1)].drop_duplicates())
high_corr_df.index

## Visualization 

In [ ]:
plt.hist(Data_copy['target'])
plt.title('Target Distribution', size=20)
plt.show()

In [ ]:
num_feature = Data_copy.columns[Data_copy.dtypes!='object']

def my_plot(feature):
    plt.hist(Data_copy[feature])
    plt.title(feature)
    plt.show()
        
for i in num_feature:
    my_plot(i)

## Prepare Data For Training

In [ ]:
Y_train = Data['target']
Y_train = pd.DataFrame(Y_train)

In [ ]:
Y_train

In [ ]:
plt.figure(figsize=(8,6))
plt.boxplot(Data_copy['target'], flierprops=dict(markerfacecolor='b', marker='o'), showfliers=True, vert=True)
plt.title('Target Box Plot', size=16)
plt.show()

In [ ]:
Data = Data.drop(["target"],axis=1 )

## Training Phase

In [ ]:
x_train, x_test, y_train, y_test = train_test_split(Data, Y_train, test_size=0.2, random_state=0)

In [ ]:
lr_model = LinearRegression()
lr_model.fit(x_train, y_train)
y_pred = lr_model.predict(x_test)

In [ ]:
y_pred

In [ ]:
mae = mean_absolute_error(y_test, y_pred)
rmse = np.sqrt(mean_squared_error(y_test, y_pred))

print('MAE = ', mae.round(4))
print('RMSE = ', rmse.round(4))

## Test Phase

In [ ]:
for i in test:
    if(i.find("cat")!= -1):
        Encoding = OrdinalEncoder()
        test[i]=Encoding.fit_transform(test[i].values.reshape(-1,1))

In [ ]:
test

In [ ]:
test_y_pred = lr_model.predict(test)
test_y_pred

In [ ]:
output = pd.DataFrame()
output['id']=test.id
output['target']=test_y_pred
output.to_csv('submission.csv')
output.set_index('id').to_csv('submission.csv')